# JetRacer Cloud Trainer 

このノートブックは、Google ColabのGPUを利用してJetRacerのAIモデルを学習するためのツールです。

**主な機能:**
- **インタラクティブUI:** ボタンやスライダーで直感的に操作できます。学習率（Learning Rate）の変更も行えます
- **ダイレクトアップロード:** Google Driveを経由せず、直接データセット（zipファイル）をアップロードできます。
- **学習結果のCSV出力:** 学習の進捗をCSVファイルに保存し、後からグラフ化や分析が可能です。
- **リアルタイム進捗表示:** 学習状況や動画の作成状況をリアルタイムで確認できます。

### **ステップ0: 必要なパッケージのインストール**

最初に、モデルの学習に必要なPythonスクリプトをダウンロードします。
（カスタムしている場合は、直接2つのファイルをuploadしてください）

In [ ]:
!wget -q https://raw.githubusercontent.com/NVIDIA-AI-IOT/jetracer/master/notebooks/utils.py
!wget -q https://raw.githubusercontent.com/NVIDIA-AI-IOT/jetracer/master/notebooks/xy_dataset.py
print("✅ 準備が完了しました。")

### **ステップ1: データセットの準備**

下の「Upload Datasets (.zip)」ボタンを使い、JetRacerで収集したデータセットのzipファイルをアップロードしてください。複数ファイルの同時アップロードも可能です。

アップロードが完了すると、ファイルが自動的に展開され、下のドロップダウンメニューに追加されます。

In [ ]:
import ipywidgets as widgets
from google.colab import files
import zipfile
import io
import os
import shutil
from IPython.display import display, clear_output

# --- UIウィジェット ---
upload_button = widgets.Button(
    description='Upload Datasets (.zip)',
    button_style='primary',
    tooltip='Click to upload zip files containing datasets',
    icon='upload'
)
upload_output = widgets.Output() # ログや処理状況を表示するエリア
dataset_list_widget = widgets.SelectMultiple(
    options=[],
    description='学習に使用するデータセット:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='150px')
)

# --- 関数 ---
def find_xy_path(root_dir):
    """指定されたディレクトリ内を再帰的に検索し、'xy'フォルダを含むディレクトリのパスを返す"""
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if 'xy' in dirnames:
            return dirpath
    return None

def on_upload_button_clicked(b):
    with upload_output:
        clear_output() # 以前のログをクリア
        print("ファイル選択ダイアログを開きます。アップロードするzipファイルを選択してください。")
        
        uploaded = files.upload()
        
        if not uploaded:
            print("ファイルがアップロードされませんでした。")
            return
            
        print("\nアップロード処理中...")
        
        current_options = list(dataset_list_widget.options)
        
        for name, content in uploaded.items():
            try:
                with zipfile.ZipFile(io.BytesIO(content), 'r') as zf:
                    extract_base_path = os.path.join('/content/', os.path.splitext(name)[0])
                    if os.path.exists(extract_base_path):
                        shutil.rmtree(extract_base_path) # 古いディレクトリを削除
                    os.makedirs(extract_base_path)
                    
                    zf.extractall(extract_base_path)
                    
                    # 展開されたディレクトリ内で'xy'フォルダを持つパスを探索
                    final_path = find_xy_path(extract_base_path)

                    if final_path and final_path not in current_options:
                        current_options.append(final_path)
                        print(f"✅ '{name}' を展開し、データセットパス '{final_path}' を検出しました。")
                    elif final_path:
                         print(f"ℹ️ '{name}' は既に追加されています。")
                    else:
                        print(f"❌ '{name}' を展開しましたが、'xy'ディレクトリが見つかりませんでした。zipの構造を確認してください。")

            except Exception as e:
                print(f"❌ '{name}' の処理中にエラーが発生しました: {e}")
        
        dataset_list_widget.options = sorted(current_options)
        dataset_list_widget.value = tuple(sorted(current_options))
        print("\nデータセットリストを更新しました。")

upload_button.on_click(on_upload_button_clicked)

display(widgets.VBox([
    widgets.HBox([upload_button]), 
    upload_output, 
    dataset_list_widget
]))

### **ステップ2: モデルの学習**

上のリストから学習に使用したいデータセットを選択し、エポック数とバッチサイズを設定して「学習開始」ボタンを押してください。
- **Epochs:** データセット全体を何回繰り返し学習するか。
- **Batch Size:** 一度に何枚の画像をまとめて処理するか。

学習が完了すると、最も性能の良かったモデル `cloud_best_model.pth` と、学習記録 `learning_log.csv` が生成されます。

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, DataLoader, random_split, Subset
import torch.optim as optim
import pandas as pd
import re
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

from xy_dataset import XYDataset

# --- モデル定義と学習関数 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = None
optimizer = None
output_dim = 4 # xy and speed (2*2)

def get_model(model_path=None):
    global output_dim
    weights = 'DEFAULT' if not model_path else None
    current_model = torchvision.models.resnet18(weights=weights)
    current_model.fc = torch.nn.Linear(512, output_dim)
    if model_path and os.path.exists(model_path):
        print(f"'{model_path}' から学習済みの重みを読み込みます...")
        current_model.load_state_dict(torch.load(model_path))
    else:
        print("ResNet-18の学習済みモデルから新規に学習を開始します...")
    return current_model.to(device)

def train_model(b):
    global model, optimizer
    train_button.disabled = True
    train_log_widget.value = "学習準備中...\n"
    with graph_output_live:
        clear_output()

    # データセットの準備
    selected_datasets = dataset_list_widget.value
    if not selected_datasets:
        train_log_widget.value = "エラー: 学習するデータセットが選択されていません。"
        train_button.disabled = False
        return
        
    all_datasets = []
    train_log_widget.value += "データセットを読み込んでいます...\n"
    for path in selected_datasets:
        try:
            dataset = XYDataset(path, ['xy', 'speed'], 
                transforms.Compose([
                    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2), #(brightness=0.5, contrast=0.2, saturation=0.2, hue=0.2)これだと輝度のばらつきが改善するかも
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]), random_hflip=True)
            all_datasets.append(dataset)
            train_log_widget.value += f"- '{path}' ({len(dataset)}件) 読み込み完了\n"
        except Exception as e:
             train_log_widget.value += f"- 警告: '{path}' の読み込み失敗: {e}\n"
    
    if not all_datasets:
        train_log_widget.value += "\nエラー: 有効なデータセットがありません。"
        train_button.disabled = False
        return

    full_dataset = ConcatDataset(all_datasets)
    
    # データの分割
    test_split = 0.1
    test_size = int(len(full_dataset) * test_split)
    train_size = len(full_dataset) - test_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_widget.value, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_widget.value, shuffle=False, num_workers=2)
    
    # モデルとオプティマイザの初期化
    model = get_model()
    optimizer = optim.Adam(model.parameters())
    
    best_loss = float('inf')
    epochs = epochs_widget.value
    log_data = []

    train_log_widget.value = f"学習を開始します (Total: {len(full_dataset)}, Train: {train_size}, Test: {test_size})\n"
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, category_idx, xy in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            images, xy = images.to(device), xy.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = 0.0
            for i, cat_idx in enumerate(category_idx.flatten()):
                loss += torch.mean((outputs[i][2*cat_idx:2*cat_idx+2] - xy[i])**2)
            loss /= len(category_idx)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for images, category_idx, xy in tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Test]"):
                images, xy = images.to(device), xy.to(device)
                outputs = model(images)
                loss = 0.0
                for i, cat_idx in enumerate(category_idx.flatten()):
                    loss += torch.mean((outputs[i][2*cat_idx:2*cat_idx+2] - xy[i])**2)
                loss /= len(category_idx)
                test_loss += loss.item()
                
        avg_train_loss = train_loss / len(train_loader)
        avg_test_loss = test_loss / len(test_loader)
        
        log_data.append({
            'epoch': epoch + 1,
            'train_loss': avg_train_loss,
            'test_loss': avg_test_loss
        })
        
        train_log_widget.value += f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.5f}, Test Loss: {avg_test_loss:.5f}\n"
        train_progress_widget.value = (epoch + 1) / epochs
        
        if avg_test_loss < best_loss:
            best_loss = avg_test_loss
            torch.save(model.state_dict(), 'cloud_best_model.pth')
            train_log_widget.value += f"  -> ✨ New best model saved with loss: {best_loss:.5f}\n"
        
        # 各エポック終了時にグラフを更新
        with graph_output_live:
            clear_output(wait=True)
            df_live = pd.DataFrame(log_data)
            plt.style.use('seaborn-v0_8-whitegrid')
            plt.figure(figsize=(10, 6))
            plt.plot(df_live['epoch'], df_live['train_loss'], 'o-', label='Train Loss')
            plt.plot(df_live['epoch'], df_live['test_loss'], 'o-', label='Test Loss')
            plt.title(f"Learning Curve (Train: {train_size} / Test: {test_size})")
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.legend()
            plt.grid(True)
            plt.show()

    # CSVログの保存
    df = pd.DataFrame(log_data)
    df.to_csv('learning_log.csv', index=False)
    
    train_log_widget.value += "\n✅ 学習が完了しました。\n"
    train_log_widget.value += "'cloud_best_model.pth' と 'learning_log.csv' が保存されました。\n"
    train_button.disabled = False
    download_button.disabled = False
    csv_download_button.disabled = False

# --- UIウィジェット (学習) ---
epochs_widget = widgets.IntText(description='Epochs', value=30, layout=widgets.Layout(width='150px'))
batch_widget = widgets.IntText(description='Batch Size', value=8, layout=widgets.Layout(width='150px'))
train_button = widgets.Button(description='学習開始', button_style='success')
train_progress_widget = widgets.FloatProgress(min=0.0, max=1.0, description='Progress')
train_log_widget = widgets.Textarea(layout=widgets.Layout(width='100%', height='250px'))
graph_output_live = widgets.Output() # リアルタイムグラフ表示用

# --- イベントリスナー (学習) ---
train_button.on_click(train_model)

# --- UI表示 (学習) ---
display(widgets.VBox([
    widgets.HBox([epochs_widget, batch_widget]), 
    train_button, 
    train_progress_widget, 
    train_log_widget,
    graph_output_live
]))

### **ステップ3: 学習結果のダウンロード**

学習で生成されたベストモデルと学習ログ（CSV）をダウンロードします。

In [ ]:
from google.colab import files

download_button = widgets.Button(
    description="Download Model (.pth)", 
    button_style='info',
    disabled=True
)
csv_download_button = widgets.Button(
    description="Download Log (.csv)", 
    button_style='info',
    disabled=True
)

def download_model(b):
    if os.path.exists('cloud_best_model.pth'):
        files.download('cloud_best_model.pth')
    else:
        print("モデルファイルが見つかりません。")

def download_csv(b):
    if os.path.exists('learning_log.csv'):
        files.download('learning_log.csv')
    else:
        print("CSVログファイルが見つかりません。")

download_button.on_click(download_model)
csv_download_button.on_click(download_csv)

display(widgets.HBox([download_button, csv_download_button]))

### **ステップ3.5: 学習曲線の可視化**

ステップ2で保存された`learning_log.csv`ファイルを元に、学習の進捗をグラフで表示します。
「グラフを表示」ボタンを押してください。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- UIウィジェット ---
show_graph_button = widgets.Button(
    description="学習グラフを表示",
    button_style='success'
)
graph_output = widgets.Output() # グラフ描画用の出力エリア

# --- 関数 ---
def show_learning_curve(b):
    with graph_output:
        clear_output(wait=True)
        log_file = 'learning_log.csv'
        if not os.path.exists(log_file):
            print(f"エラー: {log_file} が見つかりません。先にステップ2の学習を実行してください。")
            return
        
        try:
            # CSVファイルを読み込む
            df = pd.read_csv(log_file)
            
            # グラフの描画
            plt.style.use('seaborn-v0_8-whitegrid')
            fig, ax = plt.subplots(figsize=(10, 6))
            
            ax.plot(df['epoch'], df['train_loss'], marker='o', linestyle='-', label='Train Loss')
            ax.plot(df['epoch'], df['test_loss'], marker='o', linestyle='-', label='Test Loss')
            
            ax.set_title('Learning Curve', fontsize=16)
            ax.set_xlabel('Epoch', fontsize=12)
            ax.set_ylabel('Loss', fontsize=12)
            ax.legend(fontsize=12)
            ax.set_xticks(df['epoch'][::max(1, len(df)//10)]) # X軸の目盛りを間引いて表示
            
            # Test Lossが最小のポイントをマーク
            min_test_loss_epoch = df.loc[df['test_loss'].idxmin()]
            ax.annotate(f"Best Model\nEpoch: {int(min_test_loss_epoch['epoch'])}\nLoss: {min_test_loss_epoch['test_loss']:.4f}",
                        xy=(min_test_loss_epoch['epoch'], min_test_loss_epoch['test_loss']),
                        xytext=(min_test_loss_epoch['epoch'], min_test_loss_epoch['test_loss'] + 0.02),
                        arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=8),
                        bbox=dict(boxstyle="round,pad=0.3", fc="yellow", ec="black", lw=1, alpha=0.7))

            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"グラフの描画中にエラーが発生しました: {e}")

# --- イベントリスナー ---
show_graph_button.on_click(show_learning_curve)

# --- UI表示 ---
display(widgets.VBox([show_graph_button, graph_output]))

### **ステップ4: 評価動画の作成**

学習したモデルの性能を視覚的に確認するための動画を作成します。
まず、評価に使用するデータセット（学習に使っていないデータが望ましい）をアップロードしてください。その後、ドロップダウンからデータセットを選択し、「動画を作成」ボタンを押します。

In [ ]:
import cv2
import glob
from utils import preprocess
from IPython.display import HTML
from base64 import b64encode
from google.colab import files
import ipywidgets as widgets
import os
import zipfile
import io
import shutil

# --- グローバル変数 ---
generated_video_path = None

# --- UIウィジェット (評価) ---
eval_upload_button = widgets.Button(
    description='Upload Eval Data (.zip)',
    button_style='primary',
    tooltip='Click to upload zip files for evaluation',
    icon='upload'
)
eval_upload_output = widgets.Output()
video_dataset_widget = widgets.Dropdown(options=[], description='データセット:')
video_name_widget = widgets.Text(description='動画ファイル名:', value='evaluation_video.mp4')
create_video_button = widgets.Button(description='動画を作成', button_style='success')
video_log_widget = widgets.Textarea(layout=widgets.Layout(width='100%', height='150px'))
video_player_widget = widgets.Output()
video_download_button = widgets.Button(description="Download Video (.mp4)", button_style='info', disabled=True)

# --- 関数 (評価) ---
def find_xy_path_eval(root_dir):
    """評価用のxyパス検索関数"""
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if 'xy' in dirnames:
            return dirpath
    return None

def on_eval_upload_button_clicked(b):
    with eval_upload_output:
        clear_output(wait=True)
        print("評価用のzipファイルを選択してください。")
        uploaded = files.upload()
        
        if not uploaded:
            print("ファイルがアップロードされませんでした。")
            return
            
        print("\nアップロード処理中...")
        current_options = list(video_dataset_widget.options)
        
        for name, content in uploaded.items():
            try:
                with zipfile.ZipFile(io.BytesIO(content), 'r') as zf:
                    extract_base_path = os.path.join('/content/eval/', os.path.splitext(name)[0])
                    if os.path.exists(extract_base_path):
                        shutil.rmtree(extract_base_path)
                    os.makedirs(extract_base_path)
                    
                    zf.extractall(extract_base_path)
                    
                    final_path = find_xy_path_eval(extract_base_path)

                    if final_path and final_path not in current_options:
                        current_options.append(final_path)
                        print(f"✅ '{name}' を展開し、データセットパス '{final_path}' を検出しました。")
                    elif final_path:
                        print(f"ℹ️ '{name}' は既に追加されています。")
                    else:
                        print(f"❌ '{name}' を展開しましたが 'xy' ディレクトリが見つかりませんでした。")

            except Exception as e:
                print(f"❌ '{name}' の処理中にエラーが発生しました: {e}")
        
        video_dataset_widget.options = sorted(current_options)
        print("\n評価用データセットのリストを更新しました。")

def create_video(b):
    global generated_video_path
    create_video_button.disabled = True
    video_log_widget.value = "動画作成を開始します...\n"

    if not os.path.exists('cloud_best_model.pth'):
        video_log_widget.value += "エラー: モデルファイル 'cloud_best_model.pth' が見つかりません。先に学習を実行してください。"
        create_video_button.disabled = False
        return

    eval_model = get_model('cloud_best_model.pth')
    eval_model.eval()

    dataset_path = video_dataset_widget.value
    if not dataset_path:
        video_log_widget.value += "エラー: 評価するデータセットを選択してください。"
        create_video_button.disabled = False
        return

    image_dir = os.path.join(dataset_path, 'xy')
    if not os.path.exists(image_dir):
        video_log_widget.value += f"エラー: ディレクトリ '{image_dir}' が見つかりません。"
        create_video_button.disabled = False
        return
        
    image_files = sorted(glob.glob(os.path.join(image_dir, '*.jpg')), key=os.path.getmtime)
    if not image_files:
        video_log_widget.value += "エラー: 画像ファイルが見つかりません。"
        create_video_button.disabled = False
        return

    output_path = os.path.join('/content/', video_name_widget.value)
    generated_video_path = output_path
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (224, 224))

    for i, image_path in enumerate(tqdm(image_files, desc="動画作成中")):
        img = cv2.imread(image_path)
        img_resized = cv2.resize(img, (224, 224))
        with torch.no_grad():
            preprocessed_img = preprocess(img_resized).to(device)
            output = eval_model(preprocessed_img).detach().cpu().numpy().flatten()
        
        x = (output[0] / 2.0 + 0.5) * 224
        y = (output[1] / 2.0 + 0.5) * 224
        cv2.circle(img_resized, (int(x), int(y)), 8, (255, 0, 0), -1)
        out.write(img_resized)

    out.release()
    video_log_widget.value += f"\n✅ 動画の作成が完了しました: {output_path}\n"
    
    # 動画プレイヤーの更新
    with video_player_widget:
        clear_output(wait=True)
        mp4 = open(output_path,'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        display(HTML(f'<video width=400 controls><source src="{data_url}" type="video/mp4"></video>'))
        
    create_video_button.disabled = False
    video_download_button.disabled = False

def download_video(b):
    if generated_video_path and os.path.exists(generated_video_path):
        files.download(generated_video_path)
    else:
        print("ビデオファイルが見つかりません。")

# --- イベントリスナー (評価) ---
eval_upload_button.on_click(on_eval_upload_button_clicked)
create_video_button.on_click(create_video)
video_download_button.on_click(download_video)

# --- UI表示 (評価) ---
eval_ui = widgets.VBox([
    eval_upload_button,
    eval_upload_output,
    widgets.HBox([video_dataset_widget, video_name_widget]),
    create_video_button,
    video_log_widget,
    video_player_widget,
    video_download_button
])
display(eval_ui)